<a href="https://colab.research.google.com/github/bhattacharjee/mtu-nlp-assignment/blob/main/assignment2/notebooks/Augmentation_wip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torchvision==0.8.2  -q -q -q
!pip install torchtext==0.8.1    -q -q -q
!pip install torchaudio==0.7.2   -q -q -q
!pip install torch==1.7.1        -q -q -q
!pip install tqdm==4.49.0        -q -q -q
!pip install transformers==4.2.2 -q -q -q
!pip install tensorflow          -q -q -q
!pip install tensorboard         -q -q -q
!pip install tensorboardX        -q -q -q
!pip install --upgrade virtualenv -q -q -q
!pip install sentencepiece==0.1.94       -q -q -q
!pip install mosestokenizer==1.1.0 -q -q -q
!pip install tqdm

!rm -rf mtu-nlp-assignment
!git clone https://github.com/bhattacharjee/mtu-nlp-assignment.git

Cloning into 'mtu-nlp-assignment'...
remote: Enumerating objects: 1021, done.
remote: Counting objects: 100% (1021/1021), done.
remote: Compressing objects: 100% (886/886), done.
remote: Total 1021 (delta 503), reused 383 (delta 131), pack-reused 0
Receiving objects: 100% (1021/1021), 96.09 MiB | 36.78 MiB/s, done.
Resolving deltas: 100% (503/503), done.


# BackTranslation Implementation

In [4]:
from transformers import MarianMTModel, MarianTokenizer
import torch

if torch.cuda.is_available():
    print("Using cuda")
    torch_device = 'cuda'
    cuda = torch.device('cuda')
    print(cuda)
    #torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch_device = 'cpu'

target_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
target_model = MarianMTModel.from_pretrained(target_model_name)

en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
en_tokenizer = MarianTokenizer.from_pretrained(en_model_name)
en_model = MarianMTModel.from_pretrained(en_model_name)

if torch.cuda.is_available():
    target_model = target_model.to(torch_device)
    en_model = en_model.to(torch_device)

def translate(textarray, model, tokenizer, lang="fr"):
    # Prepare the text data into appropriate format for the model
    def templatize(text):
        if lang== 'en':
            return text
        else:
            return f">>{lang}<< {text}"
    textarray = [templatize(t) for t in textarray]
    encoded = tokenizer.prepare_seq2seq_batch(textarray, return_tensors='pt').to(torch_device)
    translated = model.generate(**encoded).to(torch_device)
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return translated_texts

def back_translate(texts, source_lang="en", target_lang="fr"):
    # Translate from source to target language
    if isinstance(texts, str):
        texts = [texts]
    fr_texts = translate(texts, target_model, target_tokenizer, 
                         lang=target_lang)
    # Translate from target language back to source language
    back_translated_texts = translate(fr_texts, en_model, en_tokenizer, 
                                      lang=source_lang)
    return back_translated_texts

In [ ]:
import time
t1 = time.time()
arr = ["Return the time in seconds since the epoch as a floating point number. The specific date of the epoch and the handling of leap seconds is platform dependent. On Windows and most Unix systems, the epoch is January 1, 1970, 00:00:00 (UTC) and leap seconds are not counted towards the time in seconds since the epoch. This is commonly referred to as Unix time. To find out what the epoch is on a given platform, look at gmtime(0)."]
for i in range(5):
    translated = back_translate(arr)
print(f"Time taken: {time.time() - t1} seconds")